In [1]:
pip install pinecone

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pinecone import Pinecone


In [2]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="pcsk_5RSTcB_8GfPp22T2dhbtGz59ihsbPZQnG6HHnia2QJosnBy22T8ZTu1wdcwhqdWqhRrUcm")

pc.create_index(
  name="teste",
  dimension=1024,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

{
    "name": "teste",
    "metric": "cosine",
    "host": "teste-kg31ntv.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [3]:
index = pc.Index(
    name="teste" # <-- make sure to set these
)

In [7]:
import cohere
from pinecone import Pinecone



# 1. CONEXÕES
co = cohere.Client(api_key="RAjqpE9uTVFfTTaF0RIGUM4PaKBeQ7insNIu9KLC") # Cliente do Cohere

# Conecta ao seu índice
index = pc.Index("teste") # Use o nome do seu índice

# 2. SEUS DOCUMENTOS (os "chunks" ou pedaços)
textos_dos_documentos = [
    "A Lei Geral de Proteção de Dados (LGPD) tem como principal objetivo proteger os direitos fundamentais de liberdade e de privacidade...",
    "Um dos maiores desafios da LGPD foi a necessidade de as empresas adaptarem seus processos internos...",
    "O Marco Civil da Internet estabelece princípios, garantias, direitos e deveres para o uso da Internet no Brasil.",
    "o nome do personagem é Pedro."
]

# 3. GERAR EMBEDDINGS E FAZER UPSERT
response = co.embed(
    texts=textos_dos_documentos,
    model='embed-multilingual-v3.0', # Modelo para múltiplos idiomas
 
    input_type='search_document' 
)

embeddings = response.embeddings

# Preparar os dados para o upsert
vectors_to_upsert = []
for i, embedding in enumerate(embeddings):
    vectors_to_upsert.append({
        "id": f"doc-chunk-{i}", # Crie IDs únicos
        "values": embedding,
        "metadata": {"texto": textos_dos_documentos[i]}
    })
index.delete(delete_all=True)

# Fazer o upsert em lote
print(f"Enviando lote de {len(vectors_to_upsert)} vetores...")
index.upsert(vectors=vectors_to_upsert)

print("Indexação com Cohere Embeddings concluída!")
print(index.describe_index_stats())

Enviando lote de 4 vetores...
Indexação com Cohere Embeddings concluída!
{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 0}},
 'total_vector_count': 0}
